## Analysis of Asthma

This notebook applies the `Get_Job_Script.ipynb` to automatically generate the sbatch scripts to run in Yale's cluster. The end result is to apply [various LMM workflows](https://github.com/statgenetics/UKBB_GWAS_dev/tree/master/workflow) to perform association analysis in the ASTHMA trait, do clumping analysis and extract associated regions.

## File paths on Yale cluster

- Genotype files in PLINK format:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/UKB_Caucasians_phenotypeindepqc120319_updated020720removedwithdrawnindiv`
- Genotype files in bgen format:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb39554_imputeddataset/`
- Summary stats for imputed variants BOLT-LMM:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data`
- Summary stats for inputed variants FastGWA:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data`
- Phenotype files:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/pleiotropy_R01/phenotypesforanalysis`
- Relationship file:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/unrelated_n307259/UKB_unrelatedcauc_phenotypes_asthmat2dbmiwaisthip_agesex_waisthipratio_040620`

## File paths to specific phenotypic files for Asthma:

This were the ones used in the analysis prior to the full pipeline implementation

```
phenoFile=~/project/phenotypes_UKB/Asthma_casesbyICD10codesANDselfreport_controlsbyselfreportandicd10_noautoimmuneincontrols_fastGWA.phe
covarFile=~/project/phenotypes_UKB/Asthma_casesbyICD10codesANDselfreport_controlsbyselfreportandicd10_noautoimmuneincontrols_fastGWA_covSEX.txt
qcovarFile=~/project/phenotypes_UKB/Asthma_casesbyICD10codesANDselfreport_controlsbyselfreportandicd10_noautoimmuneincontrols_fastGWA_covAGE.txt
```

## 07/01/20 analysis

On the cluster, open up this notebook using the JupyterLab server you set up via the ssh channel, then run the following cells,

## Bash variables for workflow configuration

In [2]:
# Common variables
tpl_file=../farnam.yml
bfile=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/UKB_Caucasians_phenotypeindepqc120319_updated082020removedwithdrawnindiv.bed
sampleFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb39554_imputeddataset/ukb32285_imputedindiv.sample
bgenFile=`echo /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb39554_imputeddataset/ukb_imp_chr{1..22}_v3.bgen`
unrelated_samples=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/unrelated_n307259/UKB_unrelatedcauc_phenotypes_asthmat2dbmiwaisthip_agesex_waisthipratio_040620
formatFile_fastgwa=~/project/UKBB_GWAS_dev/data/fastGWA_template.yml
formatFile_bolt=~/project/UKBB_GWAS_dev/data/boltlmm_template.yml
formatFile_saige=~/project/UKBB_GWAS_dev/data/saige_template.yml
container_lmm=/gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif
container_marp=/gpfs/gibbs/pi/dewan/data/UKBiobank/marp.sif
# LMM directories
lmm_dir_fastgwa=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/asthma
lmm_dir_bolt=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data/asthma
lmm_dir_saige=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/SAIGE_results/results_imputed_data/asthma
lmm_sos=~/project/bioworkflows/GWAS/LMM.ipynb
lmm_sbatch_fastgwa=../output/$(date +"%Y-%m-%d")_asthma-fastgwa.sbatch
lmm_sbatch_bolt=../output/$(date +"%Y-%m-%d")_asthma-bolt.sbatch
lmm_sbatch_saige=../output/$(date +"%Y-%m-%d")_asthma-saige.sbatch
phenoFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/pleiotropy_R01/phenotypesforanalysis/Asthma_casesbyICD10codesANDselfreport_controlsbyselfreportandicd10_noautoimmuneincontrols_forbolt030720
## LMM variables 
covarFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/pleiotropy_R01/phenotypesforanalysis/Asthma_casesbyICD10codesANDselfreport_controlsbyselfreportandicd10_noautoimmuneincontrols_forbolt030720
LDscoresFile=~/software/BOLT-LMM_v2.3.4/tables/LDSCORE.1000G_EUR.tab.gz
geneticMapFile=~/software/BOLT-LMM_v2.3.4/tables/genetic_map_hg19_withX.txt.gz
phenoCol=ASTHMA
covarCol=SEX
covarMaxLevels=10
qCovarCol=AGE
numThreads=20
bgenMinMAF=0.001
bgenMinINFO=0.8
lmm_job_size=1
ylim=0
### Specific to FastGWA
grmFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/UKB_Caucasians_phenotypeindepqc120319_updated020720removedwithdrawnindiv.grm.sp
### Specific to SAIGE
bgenMinMAC=4
trait_type=binary
loco=TRUE
sampleCol=IID
# LD clumping directories
clumping_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/LD_clumping/asthma
clumping_sos=~/project/bioworkflows/GWAS/LD_Clumping.ipynb
clumping_sbatch=../output/$(date +"%Y-%m-%d")_asthma_ldclumping.sbatch
## LD clumping variables
bfile_ref=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/LD_clumping/UKB_Caucasians_phenotypeindepqc120319_updated020720removedwithdrawnindiv.1210.ref_geno.bed
# For sumtastsFiles if more than one provide each path
# In this case asthma and INT-WHR
sumstatsFiles="/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/asthma/ukb_imp_v3.Asthma_casesbyICD10codesANDselfreport_controlsbyselfreportandicd10_noautoimmuneincontrols.fastGWA.snp_stats.all_chr.gz \
              /gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data/INT-WHR/ukb_imp_v3.UKB_caucasians_BMIwaisthip_AsthmaAndT2D_INT-WHR_withagesex_042020.BoltLMM.snp_stats.all_chr_modif.gz \
              /gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/T2D/diabetes_casesbyICD10andselfreport_controlswithoutautoiummune_030720_T2D.fastGWA.snp_stats.gz"
ld_sample_size=1210
clump_field=P
clump_p1=5e-08
clump_p2=1
clump_r2=0.2
clump_kb=2000
clump_annotate=BP
numThreads=20
clump_job_size=1
clumpFile=$clumping_dir/asthma_INT-WHR_T2D_ukbb.clumped
# Region extraction directories
extract_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/region_extraction/asthma
extract_sos=~/project/bioworkflows/GWAS/Region_Extraction.ipynb
extract_sbatch=../output/$(date +"%Y-%m-%d")_asthma-region.sbatch
## Region extraction variables
region_file=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/LD_clumping/asthma/asthma_INT-WHR_T2D_ukbb.clumped_region
geno_path=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/UKBB_bgenfilepath.txt
sumstats_path=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/asthma/ukb_imp_v3.Asthma_casesbyICD10codesANDselfreport_controlsbyselfreportandicd10_noautoimmuneincontrols.fastGWA.snp_stats.all_chr.gz
extract_job_size=10

## BoltLMM job

In [2]:
lmm_args="""boltlmm
    --cwd $lmm_dir_bolt 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_bolt 
    --covarFile $covarFile 
    --LDscoresFile $LDscoresFile 
    --geneticMapFile $geneticMapFile 
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --job_size $lmm_job_size
    --ylim $ylim
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_bolt \
    --args "$lmm_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2020-07-17_asthma-bolt.sbatch
INFO: Workflow farnam (ID=e70247f4a0182528) is executed successfully with 1 completed step.


## FastGWA job

In [3]:
lmm_args="""fastGWA
    --cwd $lmm_dir_fastgwa 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_fastgwa 
    --covarFile $covarFile  
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --job_size $lmm_job_size
    --grmFile $grmFile
    --ylim $ylim
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_fastgwa \
    --args "$lmm_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2020-09-15_asthma-fastgwa.sbatch
INFO: Workflow farnam (ID=d20de1f960d9c797) is executed successfully with 1 completed step.



## SAIGE job

In [ ]:
lmm_args="""SAIGE
    --cwd $lmm_dir_saige 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_saige 
    --covarFile $covarFile 
    --LDscoresFile $LDscoresFile 
    --geneticMapFile $geneticMapFile 
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --bgenMinMAC $bgenMinMAC
    --trait_type $trait_type
    --loco $loco
    --sampleCol $sampleCol
    --job_size $lmm_job_size
    --ylim $ylim
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_saige \
    --args "$lmm_args"

## LD clumping job

In [2]:
clumping_args="""default 
    --cwd $clumping_dir 
    --bfile $bfile
    --bfile_ref $bfile_ref 
    --bgenFile $bgenFile
    --sampleFile $sampleFile 
    --sumstatsFiles $sumstatsFiles 
    --unrelated_samples $unrelated_samples 
    --ld_sample_size $ld_sample_size 
    --clump_field $clump_field
    --clump_p1 $clump_p1 
    --clump_p2 $clump_p2 
    --clump_r2 $clump_r2 
    --clump_kb $clump_kb 
    --clump_annotate $clump_annotate 
    --numThreads $numThreads 
    --job_size $clump_job_size
    --clumpFile $clumpFile
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $clumping_sos \
    --to-script $clumping_sbatch \
    --args "$clumping_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2020-08-04_asthma_ldclumping.sbatch
INFO: Workflow farnam (ID=62980a243f7f8e8c) is executed successfully with 1 completed step.



## Region extract job

In [5]:
extract_args="""default_1
    --cwd $extract_dir
    --region-file $region_file
    --pheno-path $phenoFile
    --geno-path $geno_path
    --bgen-sample-path $sampleFile
    --sumstats-path $sumstats_path
    --format-config-path $formatFile_fastgwa
    --unrelated-samples $unrelated_samples
    --job-size $extract_job_size
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $extract_sos \
    --to-script $extract_sbatch \
    --args "$extract_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2020-08-04_asthma-region.sbatch
INFO: Workflow farnam (ID=297df9a0d15e7110) is executed successfully with 1 completed step.



## Epistasis job with PLINK

In [2]:
epistasis_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/interaction/set2_chr11_chr20
bfile=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/interaction/ukb_asthma_interaction.bed
phenoFile=~/scratch60/Epistasis/asthma/Asthma_casesbyICD10codesANDselfreport_controlsbyselfreportandicd10_noautoimmuneincontrols_forbolt030720_epistasis
setFile=~/scratch60/Epistasis/asthma/set2_chr11_chr20
epistasis_sos=~/project/UKBB_GWAS_dev/workflow/Epistasis.ipynb
epistasis_sbatch=../output/$(date +"%Y-%m-%d")_asthma-set2_chr11_chr20-epistasis.sbatch
numThreads=20
job_size=1
tpl_file=../farnam.yml
container_lmm=/gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif

epistasis_args="""epistasis
    --cwd $epistasis_dir 
    --bfile $bfile
    --phenoFile $phenoFile
    --setFile $setFile
    --numThreads $numThreads
    --job_size $job_size
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $epistasis_sos \
    --to-script $epistasis_sbatch \
    --args "$epistasis_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2020-10-02_asthma-set2_chr11_chr20-epistasis.sbatch
INFO: Workflow farnam (ID=5a1b05b3001b354a) is executed successfully with 1 completed step.


In [22]:
%save chr3_filter.sh -f
#!/bin/bash
#SBATCH --partition general
#SBATCH --nodes 1
#SBATCH --ntasks-per-node 1
#SBATCH --cpus-per-task 4
#SBATCH --mem 40G
#SBATCH --time 1-0:00:00
#SBATCH --job-name ../output/chr3
#SBATCH --output ../output/chr3-%J.out
#SBATCH --error ../output/chr3-%J.log
module load PLINK/2_x86_64_20180428
plink2 \
   --bgen /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb39554_imputeddataset/ukb_imp_chr3_v3.bgen \
   --sample /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb39554_imputeddataset/ukb32285_imputedindiv.sample \
   --keep /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/unrelated_n307259/UKB_unrelatedcauc_phenotypes_asthmat2dbmiwaisthip_agesex_waisthipratio_040620 \
   --make-bed --out ~/scratch60/Epistasis/asthma/ukb_imp_chr3_v3_unrelated

Cell content saved to chr3_filter.sh, use option -r to also execute the cell.

In [11]:
%save chr3_filter_exclude.sh -f
#!/bin/bash
#SBATCH --partition general
#SBATCH --nodes 1
#SBATCH --ntasks-per-node 1
#SBATCH --cpus-per-task 4
#SBATCH --mem 40G
#SBATCH --time 1-0:00:00
#SBATCH --job-name ../output/chr3
#SBATCH --output ../output/chr3-%J.out
#SBATCH --error ../output/chr3-%J.log
module load PLINK/2_x86_64_20180428
plink2 \
   --bfile /home/dc2325/scratch60/Epistasis/asthma/ukb_imp_chr3_v3_unrelated \
   --exclude /home/dc2325/scratch60/Epistasis/asthma/ukb_asthma_interaction-merge.missnp \
   --make-bed --out ~/scratch60/Epistasis/asthma/ukb_imp_chr3_v3_unrelated_exclude

Cell content saved to chr3_filter_exclude.sh, use option -r to also execute the cell.

In [1]:
%save chr3_filter_exclude_maf.sh -f
#!/bin/bash
#SBATCH --partition general
#SBATCH --nodes 1
#SBATCH --ntasks-per-node 1
#SBATCH --cpus-per-task 4
#SBATCH --mem 40G
#SBATCH --time 1-0:00:00
#SBATCH --job-name ../output/chr3_maf
#SBATCH --output ../output/chr3_maf-%J.out
#SBATCH --error ../output/chr3_maf-%J.log
module load PLINK/2_x86_64_20180428
plink2 \
   --bfile /home/dc2325/scratch60/Epistasis/asthma/ukb_imp_chr3_v3_unrelated_exclude \
   --maf 0.05 \
   --geno 0.1 \
   --mind 0.1 \
   --make-bed --out /gpfs/gibbs/pi/dewan/data/UKBiobank/results/interaction/ukb_imp_chr3_v3_unrelated_exclude_maf

Cell content saved to chr3_filter_exclude_maf.sh, use option -r to also execute the cell.

In [19]:
%save chr11_filter.sh -f
#!/bin/bash
#SBATCH --partition general
#SBATCH --nodes 1
#SBATCH --ntasks-per-node 1
#SBATCH --cpus-per-task 4
#SBATCH --mem 40G
#SBATCH --time 1-0:00:00
#SBATCH --job-name ../output/chr11
#SBATCH --output ../output/chr11-%J.out
#SBATCH --error ../output/chr11-%J.log
module load PLINK/2_x86_64_20180428
plink2 \
   --bgen /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb39554_imputeddataset/ukb_imp_chr11_v3.bgen \
   --sample /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb39554_imputeddataset/ukb32285_imputedindiv.sample \
   --keep /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/unrelated_n307259/UKB_unrelatedcauc_phenotypes_asthmat2dbmiwaisthip_agesex_waisthipratio_040620 \
   --make-bed --out ~/scratch60/Epistasis/asthma/ukb_imp_chr11_v3_unrelated 

Cell content saved to chr11_filter.sh, use option -r to also execute the cell.

In [12]:
%save chr11_filter_exclude.sh -f
#!/bin/bash
#SBATCH --partition general
#SBATCH --nodes 1
#SBATCH --ntasks-per-node 1
#SBATCH --cpus-per-task 4
#SBATCH --mem 40G
#SBATCH --time 1-0:00:00
#SBATCH --job-name ../output/chr11
#SBATCH --output ../output/chr11-%J.out
#SBATCH --error ../output/chr11-%J.log
module load PLINK/2_x86_64_20180428
plink2 \
   --bfile /home/dc2325/scratch60/Epistasis/asthma/ukb_imp_chr11_v3_unrelated \
   --exclude /home/dc2325/scratch60/Epistasis/asthma/ukb_asthma_interaction-merge.missnp \
   --make-bed --out ~/scratch60/Epistasis/asthma/ukb_imp_chr11_v3_unrelated_exclude

Cell content saved to chr11_filter_exclude.sh, use option -r to also execute the cell.

In [2]:
%save chr11_filter_exclude_maf.sh -f
#!/bin/bash
#SBATCH --partition general
#SBATCH --nodes 1
#SBATCH --ntasks-per-node 1
#SBATCH --cpus-per-task 4
#SBATCH --mem 40G
#SBATCH --time 1-0:00:00
#SBATCH --job-name ../output/chr11_maf
#SBATCH --output ../output/chr11_maf-%J.out
#SBATCH --error ../output/chr11_maf-%J.log
module load PLINK/2_x86_64_20180428
plink2 \
   --bfile /home/dc2325/scratch60/Epistasis/asthma/ukb_imp_chr11_v3_unrelated_exclude \
   --maf 0.05 \
   --geno 0.1 \
   --mind 0.1 \
   --make-bed --out /gpfs/gibbs/pi/dewan/data/UKBiobank/results/interaction/ukb_imp_chr11_v3_unrelated_exclude_maf

Cell content saved to chr11_filter_exclude_maf.sh, use option -r to also execute the cell.

In [20]:
%save chr20_filter.sh -f
#!/bin/bash
#SBATCH --partition general
#SBATCH --nodes 1
#SBATCH --ntasks-per-node 1
#SBATCH --cpus-per-task 4
#SBATCH --mem 40G
#SBATCH --time 1-0:00:00
#SBATCH --job-name ../output/chr20
#SBATCH --output ../output/chr20-%J.out
#SBATCH --error ../output/chr20-%J.log
module load PLINK/2_x86_64_20180428
plink2 \
   --bgen /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb39554_imputeddataset/ukb_imp_chr20_v3.bgen \
   --sample /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb39554_imputeddataset/ukb32285_imputedindiv.sample \
   --keep /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/unrelated_n307259/UKB_unrelatedcauc_phenotypes_asthmat2dbmiwaisthip_agesex_waisthipratio_040620 \
   --make-bed --out ~/scratch60/Epistasis/asthma/ukb_imp_chr20_v3_unrelated 

Cell content saved to chr20_filter.sh, use option -r to also execute the cell.

In [13]:
%save chr20_filter_exclude.sh -f
#!/bin/bash
#SBATCH --partition general
#SBATCH --nodes 1
#SBATCH --ntasks-per-node 1
#SBATCH --cpus-per-task 4
#SBATCH --mem 40G
#SBATCH --time 1-0:00:00
#SBATCH --job-name ../output/chr20
#SBATCH --output ../output/chr20-%J.out
#SBATCH --error ../output/chr20-%J.log
module load PLINK/2_x86_64_20180428
plink2 \
   --bfile /home/dc2325/scratch60/Epistasis/asthma/ukb_imp_chr20_v3_unrelated \
   --exclude /home/dc2325/scratch60/Epistasis/asthma/ukb_asthma_interaction-merge.missnp \
   --make-bed --out ~/scratch60/Epistasis/asthma/ukb_imp_chr20_v3_unrelated_exclude

Cell content saved to chr20_filter_exclude.sh, use option -r to also execute the cell.

In [3]:
%save chr20_filter_exclude_maf.sh -f
#!/bin/bash
#SBATCH --partition general
#SBATCH --nodes 1
#SBATCH --ntasks-per-node 1
#SBATCH --cpus-per-task 4
#SBATCH --mem 40G
#SBATCH --time 1-0:00:00
#SBATCH --job-name ../output/chr20_maf
#SBATCH --output ../output/chr20_maf-%J.out
#SBATCH --error ../output/chr20_maf-%J.log
module load PLINK/2_x86_64_20180428
plink2 \
   --bfile /home/dc2325/scratch60/Epistasis/asthma/ukb_imp_chr20_v3_unrelated_exclude \
   --maf 0.05 \
   --geno 0.1 \
   --mind 0.1 \
   --make-bed --out /gpfs/gibbs/pi/dewan/data/UKBiobank/results/interaction/ukb_imp_chr20_v3_unrelated_exclude_maf

Cell content saved to chr20_filter_exclude_maf.sh, use option -r to also execute the cell.

In [21]:
%save chr2_filter.sh -f
#!/bin/bash
#SBATCH --partition general
#SBATCH --nodes 1
#SBATCH --ntasks-per-node 1
#SBATCH --cpus-per-task 4
#SBATCH --mem 40G
#SBATCH --time 1-0:00:00
#SBATCH --job-name ../output/chr2
#SBATCH --output ../output/chr2-%J.out
#SBATCH --error ../output/chr2-%J.log
module load PLINK/2_x86_64_20180428
plink2 \
   --bgen /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb39554_imputeddataset/ukb_imp_chr2_v3.bgen \
   --sample /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb39554_imputeddataset/ukb32285_imputedindiv.sample \
   --keep /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/unrelated_n307259/UKB_unrelatedcauc_phenotypes_asthmat2dbmiwaisthip_agesex_waisthipratio_040620 \
   --make-bed --out ~/scratch60/Epistasis/asthma/ukb_imp_chr2_v3_unrelated 

Cell content saved to chr2_filter.sh, use option -r to also execute the cell.

In [14]:
%save chr2_filter_exclude.sh -f
#!/bin/bash
#SBATCH --partition general
#SBATCH --nodes 1
#SBATCH --ntasks-per-node 1
#SBATCH --cpus-per-task 4
#SBATCH --mem 40G
#SBATCH --time 1-0:00:00
#SBATCH --job-name ../output/chr2
#SBATCH --output ../output/chr2-%J.out
#SBATCH --error ../output/chr2-%J.log
module load PLINK/2_x86_64_20180428
plink2 \
   --bfile /home/dc2325/scratch60/Epistasis/asthma/ukb_imp_chr2_v3_unrelated \
   --exclude /home/dc2325/scratch60/Epistasis/asthma/ukb_asthma_interaction-merge.missnp \
   --make-bed --out ~/scratch60/Epistasis/asthma/ukb_imp_chr2_v3_unrelated_exclude

Cell content saved to chr2_filter_exclude.sh, use option -r to also execute the cell.

In [4]:
%save chr2_filter_exclude_maf.sh -f
#!/bin/bash
#SBATCH --partition general
#SBATCH --nodes 1
#SBATCH --ntasks-per-node 1
#SBATCH --cpus-per-task 4
#SBATCH --mem 40G
#SBATCH --time 1-0:00:00
#SBATCH --job-name ../output/chr2_maf
#SBATCH --output ../output/chr2_maf-%J.out
#SBATCH --error ../output/chr2_maf-%J.log
module load PLINK/2_x86_64_20180428
plink2 \
   --bfile /home/dc2325/scratch60/Epistasis/asthma/ukb_imp_chr2_v3_unrelated_exclude \
   --maf 0.05 \
   --geno 0.1 \
   --mind 0.1 \
   --make-bed --out /gpfs/gibbs/pi/dewan/data/UKBiobank/results/interaction/ukb_imp_chr2_v3_unrelated_exclude_maf

Cell content saved to chr2_filter_exclude_maf.sh, use option -r to also execute the cell.

In [19]:
%save merge_filter.sh -f
#!/bin/bash
#SBATCH --partition general
#SBATCH --nodes 1
#SBATCH --ntasks-per-node 1
#SBATCH --cpus-per-task 4
#SBATCH --mem 60G
#SBATCH --time 1-0:00:00
#SBATCH --job-name ../output/allchr
#SBATCH --output ../output/allchr-%J.out
#SBATCH --error ../output/allchr-%J.log
#for chr in {'chr3','chr11','chr20'}; do echo "/home/dc2325/scratch60/Epistasis/asthma/ukb_imp_${chr}_v3_unrelated.bed /home/dc2325/scratch60/Epistasis/asthma/ukb_imp_${chr}_v3_unrelated.bim /home/dc2325/scratch60/Epistasis/asthma/ukb_imp_${chr}_v3_unrelated.fam\n" >> list_beds.txt; done
module load PLINK/1.90-beta5.3
plink \
  --bfile /home/dc2325/scratch60/Epistasis/asthma/ukb_imp_chr2_v3_unrelated_exclude \
  --merge-list /home/dc2325/scratch60/Epistasis/asthma/list_beds.txt \
  --make-bed --out /gpfs/gibbs/pi/dewan/data/UKBiobank/results/ukb_asthma_interaction \
  &> ../output/plink_interaction.log

Cell content saved to merge_filter.sh, use option -r to also execute the cell.

In [6]:
%save merge_filter_maf.sh -f
#!/bin/bash
#SBATCH --partition general
#SBATCH --nodes 1
#SBATCH --ntasks-per-node 1
#SBATCH --cpus-per-task 4
#SBATCH --mem 80G
#SBATCH --time 1-0:00:00
#SBATCH --job-name ../output/allchr-maf
#SBATCH --output ../output/allchr_maf-%J.out
#SBATCH --error ../output/allchr_maf-%J.log
#for chr in {'chr3','chr11','chr20'}; do echo "/gpfs/gibbs/pi/dewan/data/UKBiobank/results/interaction/ukb_imp_${chr}_v3_unrelated_exclude_maf.bed /gpfs/gibbs/pi/dewan/data/UKBiobank/results/interaction/ukb_imp_${chr}_v3_unrelated_exclude_maf.bim /gpfs/gibbs/pi/dewan/data/UKBiobank/results/interaction/ukb_imp_${chr}_v3_unrelated_exclude_maf.fam\n" >> list_beds.txt; done
module load PLINK/1.90-beta5.3
plink \
  --bfile /gpfs/gibbs/pi/dewan/data/UKBiobank/results/interaction/ukb_imp_chr2_v3_unrelated_exclude_maf \
  --merge-list /gpfs/gibbs/pi/dewan/data/UKBiobank/results/interaction/list_beds.txt \
  --make-bed --out /gpfs/gibbs/pi/dewan/data/UKBiobank/results/interaction/ukb_asthma_interaction \
  &> ../output/asthma_interaction_maf.log

Cell content saved to merge_filter_maf.sh, use option -r to also execute the cell.